comment j'ai scrappé le NYT

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# Configuration
API_KEY = "D26RTqR74kjjaopDptodzz8lKomncUTM"  # clé API NYT
QUERY = "CAC 40"
START_DATE = datetime(2000, 1, 1)  # Début de janvier 2000
END_DATE = datetime(2020, 4, 19)   # Fin de novembre 2024
OUTPUT_FILE = "mentions_cac40.csv"

# URL de l'Article Search API
BASE_URL = "https://api.nytimes.com/svc/search/v2/articlesearch.json"


# Fonction pour formater les dates au format AAAAMMJJ
def format_date(date):
    return date.strftime("%Y%m%d")


# Fonction pour récupérer les articles depuis l'API NYT
def fetch_articles(query, start_date, end_date, api_key):
    all_articles = []
    page = 0

    print("Recherche des articles en cours...")
    while True:
        # Préparer les paramètres de la requête
        params = {
            'q': query,
            'begin_date': format_date(start_date),
            'end_date': format_date(end_date),
            'api-key': api_key,
            'page': page,
            'sort': 'newest'
        }

        # Envoyer la requête
        response = requests.get(BASE_URL, params=params)
        if response.status_code != 200:
            print(f"Erreur API : {response.status_code}")
            break

        data = response.json()
        docs = data.get('response', {}).get('docs', [])

        if not docs:
            break  # Sortir si plus d'articles

        # Extraire les données nécessaires
        for article in docs:
            all_articles.append({
                'headline': article['headline']['main'],
                'snippet': article.get('snippet', ''),
                'pub_date': article.get('pub_date', ''),
                'web_url': article.get('web_url', '')
            })

        print(f"Page {page + 1} traitée ({len(docs)} articles)")
        page += 1

        if page % 5 == 0:
            print("Pause imposée par l'API...")
            time.sleep(60)

    return all_articles


# Fonction principale
def main():
    global END_DATE
    # Chemin du fichier
    file_name = "mentions_cac40.csv"

    # Tester si le fichier existe
    if os.path.isfile(file_name):
        # Charger le fichier CSV
        df = pd.read_csv("mentions_cac40.csv")

        # Convertir la colonne `pub_date` en format datetime
        df['pub_date'] = pd.to_datetime(df['pub_date'])

        # Trouver la date minimale
        derniere_date = df['pub_date'].min()

        # Convertir en datetime.date pour garder uniquement la date
        if isinstance(derniere_date, pd.Timestamp):
            derniere_date = derniere_date.to_pydatetime().date()

        # Ajouter un jour à la date
        END_DATE = derniere_date + timedelta(days=1)

    articles = fetch_articles(QUERY, START_DATE, END_DATE, API_KEY)

    if not articles:
        print("Aucun article trouvé.")
        return

    # Sauvegarder les résultats dans un fichier CSV
    df = pd.DataFrame(articles)
    df.to_csv(OUTPUT_FILE, mode='a', index=False)
    print(f"{len(articles)} articles sauvegardés dans '{OUTPUT_FILE}'.")


if __name__ == "__main__":
    main()


puis nettoyage

In [ ]:
import pandas as pd

# Charger le fichier CSV dans un DataFrame
df = pd.read_csv("mentions_cac40.csv")

# Supprimer les doublons en gardant seulement le premier article par date
df_unique = df.drop_duplicates(subset=['pub_date'], keep='first')

# Sauvegarder le résultat dans un nouveau fichier CSV
df_unique.to_csv("mentions_cac40.csv", index=False)

print("Traitement terminé. Le fichier 'articles_uniques_par_date.csv' contient les articles sans doublons par date.")


puis compter les articles

In [ ]:
import pandas as pd

input_path = 'mentions_cac40.csv'  # Chemin du fichier d'entrée
output_path = 'nbr_mentions.csv'  # Chemin du fichier de sortie

# Lire les données depuis le fichier CSV
df = pd.read_csv(input_path)

# Vérifier si la colonne 'pub_date' existe
if 'pub_date' not in df.columns:
    raise ValueError("La colonne 'pub_date' n'existe pas dans le fichier.")

# Convertir la colonne 'Date' en format datetime
try:
    df['Date'] = pd.to_datetime(df['pub_date'], errors='coerce')
except Exception as e:
    raise ValueError(f"Erreur lors de la conversion des dates : {e}")

# Identifier et retirer les valeurs non valides
invalid_dates = df[df['Date'].isna()]
if not invalid_dates.empty:
    print(f"Dates invalides détectées :\n{invalid_dates}")
    # Supprimer les lignes avec des dates non valides
    df = df.dropna(subset=['Date'])

# Définir les périodes : 1 janvier, 1 mars, 1 juin, 1 septembre
def assign_period(date):
    if date.month in [1, 2]:
        return f"{date.year}-01-01"
    elif date.month in [3, 4, 5]:
        return f"{date.year}-03-01"
    elif date.month in [6, 7, 8]:
        return f"{date.year}-06-01"
    elif date.month in [9, 10, 11]:
        return f"{date.year}-09-01"
    else:  # Décembre
        return f"{date.year}-12-01"

# Ajouter une colonne pour la période
df['Period'] = df['Date'].apply(assign_period)

# Grouper par période et compter les articles
article_counts = df.groupby('Period').size().reset_index(name='Article Count')

# Sauvegarder le résultat dans un nouveau fichier CSV
article_counts.to_csv(output_path, index=False)

print(f"Les résultats ont été sauvegardés dans : {output_path}")


après je fais une jointure avec le cac40

In [ ]:
import pandas as pd

# Charger les fichiers CSV
sortie = pd.read_csv("sortie.csv")
nbr_mentions = pd.read_csv("nbr_mentions.csv")

# Assurez-vous que les colonnes de date sont au format datetime
sortie['Date'] = pd.to_datetime(sortie['Date'], errors='coerce')
nbr_mentions['Date'] = pd.to_datetime(nbr_mentions['Date'], errors='coerce')

# Effectuer la jointure sur la colonne 'date'
resultat = pd.merge(sortie[['Date', 'Close']], nbr_mentions[['Date', 'Article Count']], on='Date', how='inner')

# Sauvegarder le fichier résultant si nécessaire
resultat.to_csv("jointure_resultat.csv", index=False)

print("La jointure a été réalisée avec succès. Le fichier est sauvegardé sous le nom 'jointure_resultat.csv'.")

puis plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_cac40_and_mentions(csv_file):
    # Load the CSV data into a DataFrame
    df = pd.read_csv(csv_file)

    # Convert the 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Create the plot
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot CAC40 Close values (left y-axis) as a line
    ax1.plot(df['Date'], df['Close'], color='blue', label='CAC40 (Close)')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('CAC40 Close', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # Create a second y-axis for Article Count
    ax2 = ax1.twinx()

    # Invert the y-axis for Article Count and plot as bars
    ax2.bar(df['Date'], df['Article Count'], color='orange', alpha=0.6, width=20, label='Article Count')
    ax2.set_ylabel('Article Count', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')
    ax2.invert_yaxis()

    # Add legends and title
    fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
    plt.title('CAC40 Close vs Article Count Over Time')

    # Show the plot
    plt.tight_layout()
    plt.show()

# Call the function with the CSV file name
plot_cac40_and_mentions('jointure_resultat.csv')
